# Requirement 1

### Structure

In [1]:
n_companies = 3
days = 10

bidding_agent=UCBlike_agent()
bidding_env=bidenv()
pricing_agent=GP_agent()
pricing_env=pricenv()
auction_env=auction(bidding_env, pricing_env)
for c in range(n_companies):
    for d in range(days):
        #choose price
        price=pricing_agent.ChoosePrice()

        #BIDDING
        bid = bidding_agent.ChooseBid(price)
        win_count = auction_env.round(bid)
        bidding_agent.update(bid,win_count)

        #PRICING
        d,r=auction_env.sell(price)
        pricing_agent.update(price,r/d)

SyntaxError: incomplete input (2836940658.py, line 16)

### This way we change bid value many times for the same price. Then each day we chamge the price p and we try with different prices